In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,161 kB]
Get:13 http://

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-23 01:07:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.36MB/s    in 0.2s    

2022-09-23 01:07:53 (5.36 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



## Reading S3 Bucket csv file to DataFrame

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Recidivism_Cloud").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://vasudha-bucket.s3.ca-central-1.amazonaws.com/recidivism_data.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+--------------------+-------------------------+-------------------------+--------------------+--------------------+---------------+------+----------------------+------------+---------------+----------------+--------------+---------------+--------------------------+----------------+--------------------+-----------------+
|Fiscal Year Released|Recidivism Reporting Year|Main Supervising District|        Release Type|    Race - Ethnicity|Age At Release |   Sex|Offense Classification|Offense Type|Offense Subtype|Return to Prison|Days to Return|Recidivism Type|New Offense Classification|New Offense Type|New Offense Sub Type|Target Population|
+--------------------+-------------------------+-------------------------+--------------------+--------------------+---------------+------+----------------------+------------+---------------+----------------+--------------+---------------+--------------------------+----------------+--------------------+-----------------+
|                2010|         

## Uploading DataFrame to AWS RDS Instance

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.clf82frcjuur.ca-central-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Database_1", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write df to prisoners table in postgres server
df.write.jdbc(url=jdbc_url, table='PRISONERS', mode=mode, properties=config)